<a href="https://colab.research.google.com/github/jhermosillo/Escuela_CD_IMATE_2019/blob/master/Wiki_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3><center>
    
## **Modelado de texto usando técnicas de reducción de dimensionalidad.**
### Aplicación en WikiPedia para medir semejanza entre documentos.
    
</center></h3>
<h5><center>
    Dr. Jorge Hermosillo Valadez<br>
    Centro de Investigación en Ciencias<br>
    Universidad Autónoma del Estado de Morelos<br>
</center></h5>
<h1><center>
<img src="img/CINC_TRANSP.png" width="100"/>
<img src="img/UAEM_COLOR_2.png" width="100"/>
</center></h1>

Descubrir temas es útil para diversos fines, como agrupar documentos, organizar contenido disponible en línea para recuperar información y hacer recomendaciones. El modelado de temas es una técnica de minería de texto que proporciona métodos para identificar palabras clave concurrentes, con el fin de resumir grandes colecciones de información textual. Ayuda a descubrir temas ocultos en el documento, anotar los documentos con estos temas y organizar una gran cantidad de datos no estructurados. Numerosos proveedores de contenido y agencias de noticias están utilizando modelos de temas para recomendar artículos a los lectores. 

Utilizaremos dos técnicas de reducción de dimensionalidad, PCA (Principal Component Analysis) y LSA (Latent Semantic Analysis), con el propósito de modelar documentos y establecer semejanzas entre ellos. 

Ambas técnicas utilizan el modelo de bolsa de palabras (BoW -- Bag of words), que da como resultado una matriz documento-término que representa documentos en función del conteo de términos. Como veremos, PCA y LSA guardan una estrecha relación. La diferencia básica es que el primero hace un pre-procesamiento de los datos mediante el centrado de las observaciones.

En este curso veremos cómo:
* Leer documentos de la wikipedia (raw)
* Construir una matriz BoW sobre de ellos
* Aplicar PCA y LSA sobre esta matriz
* Comparar el desempeño de ambos métodos analizando la semejanza entre documentos

En términos generales, el proceso que vamos a seguir es lo siguiente:

![Proceso](img/BOW.png)

# Módulos necesarios

## **Sólo para COLAB**

In [ ]:
#"""
!apt-get install subversion
!svn checkout "https://github.com/jhermosillo/Escuela_CD_IMATE_2019/trunk/datos/"
!svn checkout "https://github.com/jhermosillo/Escuela_CD_IMATE_2019/trunk/modelos/"
#"""

Para traer archivos al entorno de COLAB

In [ ]:
#"""
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
file_drive = GoogleDrive(gauth)
#"""

La siguiente instrucción requiere el vínculo al archivo desde DRIVE

In [ ]:
wi = file_drive.CreateFile({'id':'1sV6vK0CLUXpH1VInmtBUBnVACMm5fPFB'})

In [ ]:
wi.GetContentFile('wiki.py')

In [1]:
import wiki as wi

[nltk_data] Downloading package stopwords to C:\Users\Jorge
[nltk_data]     Hermosillo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [2]:
import numpy as np
import glob

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package punkt to C:\Users\Jorge
[nltk_data]     Hermosillo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


Para ahorrar tiempo...

In [ ]:
wi.mas_RAM_porfavor()

# **Cargado de archivo Wikipedia**

Dato informativo: para este curso primero descargamos los archivos raw de wikipedia (https://www.cs.upc.edu/~nlp/wikicorpus/). Estos archivos son tipo texto.

**Para propósitos de este curso, solo usamos un par de archivos cada uno con varios miles de documentos.**

In [3]:
archivos = glob.glob('./datos/textosWiki_1')
print(archivos)

['./datos/textosWiki_1']


Leemos los archivos descargados y sus nombres.

In [4]:
file,nombres = wi.carga_datos(archivos)

leyendo...
./datos/textosWiki_1
tamaño del contenido de archivos cargados:             12 MB


Verificamos algunos datos

In [5]:
print(file[0][:20],file[0][-20:])

['<doc', 'id="1871762"', 'title="jud', 'buechler"', 'nonfiltered="1"', 'processed="1"', 'dbindex="435001">', 'judson', 'donald', 'buechler', '(nacido', 'el', '19', 'de', 'junio', 'de', '1968', 'en', 'san', 'diego'] ['sostiene', 'que,', 'para', 'todas', 'las', 'personas,', 'ese', 'criterio', 'imparcial', 'se', 'aplica', 'en', 'forma', 'homogénea.', 'véase', 'también', '.', 'objetividad;', 'endofarticle.', '</doc>']


# **Extracción de documentos**

Ahora vamos a identificar cada documento y elaborar una lista de los mismos.
Para ello, se debe tener en cuenta la forma en que se indexan listas y arreglos en Python.

Como hemos visto, los archivos de Wikipedia traen el identificador de documento ```id=``` (p.ej. _id="1842224"_) y marcadores de inicio y fin de documento, que se reconocen por el caracter ```">"``` (p.ej. _dbindex="430000">_ y ```</doc>```). 

__Lo que vamos a hacer es obtener los indices en donde se encuentran estos datos para extraer el id de documento y su contenido textual__. Esto con el fin de construir una lista de documentos.

Los textos se limpian y procesan usando el módulo ```nltk``` (Natural Language ToolKit) (Loper and Bird, 2002). 

Al final tendremos una lista cuyo contenido será como sigue:

```
docs->[[id_doc1,texto del documento doc1],[id_doc2,texto del documento doc2], ... ]
```

In [6]:
docs = wi.lee_documentos(file,nombres)

G:\Mi unidad\LABORATORIO DE SEMANTICA COMPUTACIONAL\CURSOS\Escuela_CD_IMATE_2019\wiki.py:69: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  r=r.cadena.str.translate(\


archivo ./datos/textosWiki_1 contiene 4753        documentos 



In [7]:
print('Se leyeron {} archivos'.format(len(docs)))
print(docs[0][0][0],docs[0][0][1][:100])

Se leyeron 1 archivos
1871762 judson donald buechler nacido junio san diego california california unidos jugador profesional ameri


# **Data Frame de documentos**

Ahora vamos a usar pandas (McKinney & others, 2010) para crear un DataFrame que es una estructura de datos especialmente diseñada para manipular grandes cantidades de datos de manera ágil y eficiente.

In [10]:
df_0 = pd.DataFrame(docs[0],columns = ['doc_id','Texto','clase'])
#df_1 = pd.DataFrame(docs[1],columns = ['doc_id','Texto','clase'])
print(len(df_0.index),'documentos clase 0')
#print(len(df_1.index),'documentos clase 1')
df = df_0
#df = pd.concat([df_0, df_1], ignore_index=True, sort=False)
print(df.shape)
df.head()

4753 documentos clase 0
(4753, 3)


,doc_id,Texto,clase
0,1871762,judson donald buechler nacido junio san diego ...,0
1,1871768,lost highway the concert dvd recoge concierto ...,0
2,1871769,eburones tribu descendencia germánica habitaro...,0
3,1871771,aguada baixo portuguesa águeda km² área habita...,0
4,1871772,selge griego importante ciudad pisidia ladera ...,0


## **Extracción de características**

Tratemos de visualizar algunas propiedades de los documentos.

Para ello vamos a utilizar un contador ([Counter](https://docs.python.org/2/library/collections.html)). Un contador es un contenedor que almacena elementos como claves de diccionario, y sus recuentos se almacenan como valores de diccionario.

Construiremos una columna con el conteo de palabras por documento y otra con la palabra más frecuente en el documento.

Para visualizar estos datos utilizaremos [matplotlib](https://matplotlib.org/) (Hunter, 2007).

John D. Hunter. Matplotlib: A 2D Graphics Environment, _Computing in Science & Engineering, 9, 90-95 (2007)_, DOI:10.1109/MCSE.2007.55 (publisher link)

In [11]:
from collections import Counter  #regresa un diccionario con conteos

df['Palabras']=df['Texto'].apply(lambda x: x.split())
df['Total']=df['Palabras'].apply(lambda x: len(x))
df['Conteos']=df['Palabras'].apply(lambda x: Counter(x))

df=df.sort_values(by="Total",ascending=False)

df.index = range(len(df.index))
df.head()

,doc_id,Texto,clase,Palabras,Total,Conteos
0,1889137,invasiones japonesas corea conflicto bélico de...,0,"[invasiones, japonesas, corea, conflicto, béli...",7549,"{'invasiones': 6, 'japonesas': 31, 'corea': 69..."
1,1889938,cartapuebla oviedo documento concedido ciudad ...,0,"[cartapuebla, oviedo, documento, concedido, ci...",5280,"{'cartapuebla': 1, 'oviedo': 26, 'documento': ..."
2,1881683,historia sal trata uso comercio dado siglos ún...,0,"[historia, sal, trata, uso, comercio, dado, si...",4047,"{'historia': 7, 'sal': 228, 'trata': 1, 'uso':..."
3,1891268,movimiento homófilo segundo movimiento homosex...,0,"[movimiento, homófilo, segundo, movimiento, ho...",3738,"{'movimiento': 26, 'homófilo': 14, 'segundo': ..."
4,1892060,sarah trimmer enero diciembre escritora crític...,0,"[sarah, trimmer, enero, diciembre, escritora, ...",3694,"{'sarah': 16, 'trimmer': 115, 'enero': 2, 'dic..."


# **Reducción del tamaño de las matrices**

Para reducir la complejidad espacial de nuestro ejercicio, podemos hacer dos cosas:
* 1.- Un muestreo aleatorio de documentos, lo que nos ayudaría a reducir el vocabulario. 
* 2.- Un recorte en el número de documentos por la cantidad de palabras.
Usaremos el segundo.

Obtenemos el vocabulario.
Para ello vamos a usar el método de tokenización de NLTK

In [12]:
textos = df['Texto'].values
textos = " ".join(textos)
vocabulario = set(word_tokenize(textos))
print(len(vocabulario),'palabras únicas (tipos)')

103133 palabras únicas (tipos)


Filtramos algunos documentos

In [13]:
df=df[(df.Total < 2000) & (df.Total > 100)]
print(len(df))
df.index = range(len(df.index))
df.head()

2117


,doc_id,Texto,clase,Palabras,Total,Conteos
0,1886236,taifa valencia taifa balansiya reinos taifas c...,0,"[taifa, valencia, taifa, balansiya, reinos, ta...",1949,"{'taifa': 25, 'valencia': 61, 'balansiya': 4, ..."
1,1878783,deva victrix simplemente deva ciudadfortaleza ...,0,"[deva, victrix, simplemente, deva, ciudadforta...",1893,"{'deva': 19, 'victrix': 16, 'simplemente': 1, ..."
2,1882951,animax latinoamérica complejo tres canales cab...,0,"[animax, latinoamérica, complejo, tres, canale...",1847,"{'animax': 60, 'latinoamérica': 10, 'complejo'..."
3,1888749,serie fílmica superman lista largometrajes per...,0,"[serie, fílmica, superman, lista, largometraje...",1831,"{'serie': 11, 'fílmica': 2, 'superman': 75, 'l..."
4,1880333,rock rolinga llamado rock chabón rock stone ro...,0,"[rock, rolinga, llamado, rock, chabón, rock, s...",1829,"{'rock': 67, 'rolinga': 19, 'llamado': 2, 'cha..."


### Guardamos el data frame

Para ello, vamos a usar [pickle](https://docs.python.org/2/library/pickle.html), que forma parte de las [funcionalidades de I/O](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html) de Pandas.

Pickle permite serializar y deserializar una estructura de datos Python. "_Pickling_" es el proceso mediante el cual una jerarquía de objetos de Python se convierte en una secuencia de bytes, y "_Depickling_" es la operación inversa, mediante la cual una secuencia de bytes se convierte nuevamente en una jerarquía de objetos.

In [15]:
df.loc[2].Texto

'animax latinoamérica complejo tres canales cable lanzado julio señales latinoamérica brasil reemplazando canal locomotion igual filiales asia inició proyecto canal transmite únicamente series anime aunque recientemente confirmó emitira películas distribuidas sony pictures mayo estudios animax latinoamérica estan ubicados caracas venezuela igual estudios canales hermanos sony entertainment television axn pertenece sony pictures television international estructura señales complejo cuenta feeds lugares transmisión masivos cuales transmite programación señal venezuela señal brasil señal señal genérica disponible resto países latinoamérica historia siendo primer intento sony brindar canal exclusivo anime horas latinoamérica pensó poner series formatos series contuvieran capítulos transmitidas días mientras aquellas cantidad menor emitirían ciertos días semana mismo modo transmitidas japón total puede encontrar solo día capítulo estreno cualquier serie mínimo repeticiones final cada serie p

In [ ]:
df.to_pickle('datos/data_frame_4K.pkl')

<hr/>
<hr/>

# Leemos un Data Frame previamente almacenado

In [3]:
df=pd.read_pickle('datos/data_frame_4K.pkl')
df.index = range(len(df.index))
print(df.shape)
df.head()

(4041, 6)


,doc_id,Texto,clase,Palabras,Total,Conteos
0,1039186,práctica movimiento scout argentina adopta dif...,1,"[práctica, movimiento, scout, argentina, adopt...",1988,"{'práctica': 1, 'movimiento': 14, 'scout': 38,..."
1,1039418,coalición norte alianza provincias norte confe...,1,"[coalición, norte, alianza, provincias, norte,...",1987,"{'coalición': 10, 'norte': 15, 'alianza': 1, '..."
2,1886236,taifa valencia taifa balansiya reinos taifas c...,0,"[taifa, valencia, taifa, balansiya, reinos, ta...",1949,"{'taifa': 25, 'valencia': 61, 'balansiya': 4, ..."
3,1039204,elecciones municipales chile realizaron octubr...,1,"[elecciones, municipales, chile, realizaron, o...",1942,"{'elecciones': 8, 'municipales': 2, 'chile': 2..."
4,1038384,vanilla ninja banda originaria estonia compues...,1,"[vanilla, ninja, banda, originaria, estonia, c...",1911,"{'vanilla': 58, 'ninja': 58, 'banda': 26, 'ori..."


# **BoW**

Para obtener la Bolsa de Palabras, vamos a utilizar la columna de Conteos con los conteos de palabras respectivos por cada documento.

Esta columna consta de diccionarios que usaremos como entrada para el módulo [DictVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html), que transforma diccionarios en arreglos Numpy.

Antes, vamos a definir nuestra función BOW. Esta función recibe una lista de diccionarios y regresa una matriz $X$ de documentos y un diccionario $v$ con el vocabulario asociado a un entero. 

**Cada fila de $X$ es un documento y cada columna representa una palabra del vocabulario.**

<img src="img/BoW_M.png" width="300"/>

In [16]:
from sklearn.feature_extraction import DictVectorizer
from collections import Counter, OrderedDict

def bow_(docs):
    v = DictVectorizer(sparse=False)
    X = v.fit_transform(docs)
    return X,v

In [17]:
docs = df.Conteos.tolist()
X,vocab_ = bow_(docs)
print(X.shape)

(2117, 87310)


In [ ]:
print('X:')
print(X[:2])
print('La primer fila de la matriz X suma {} conteos que coincide con el primer documento.'.format(int(X[0].sum())))
print('La segunda fila de la matriz X suma {} conteos que coincide con el segundo documento.'.format(int(X[1].sum())))

# **Métodos de reducción de dimensionalidad y codificación latente**

El análisis de componentes principales (PCA) y la descomposición de valores singulares (SVD) son enfoques de reducción de dimensionalidad comúnmente utilizados en el análisis de datos exploratorios y el aprendizaje automático. 

Ambos son métodos clásicos de reducción de dimensionalidad lineal que intentan encontrar combinaciones lineales de características en la matriz de datos de alta dimensión original para construir una representación significativa del conjunto de datos.

PCA tiene como objetivo encontrar ejes ortogonales linealmente no correlacionados, que también se conocen como componentes principales (PC) en el espacio dimensional m (espacio de las características &mdash;_features_) para proyectar los puntos de datos en esas PC. La primera PC captura la mayor variación en los datos.

<img src="img/PCA.gif" width="375" height="375"/>

Las representaciones resultantes de PCA y SVD son similares para algunos datos, ya que PCA y SVD están estrechamente relacionados. Si tenemos:

<img src="img/PCA_SVD-1.png" width="75"/> 

donde, $\mathbf{x}_i=[x_1,x_2,\cdots,x_d]$ son muestras observadas de datos $d-$dimensionales, $d\in \mathbb{N}$.

Los $k$ _ejes principales_ $\mathbf{w}_j$, $j \in \{1,\cdots,k\}$, son aquellos ejes ortonormales sobre los cuales la varianza de la proyección es máxima. 

**¿Por qué la varianza?**:
Se puede considerar cualquier conjunto de observaciones como una señal contaminada con ruido. Lo deseable es maximizar la razón señal-a-ruido (SNR):
<center>
$SNR=\frac{\sigma^2_{senal}}{\sigma^2_{ruido}}$
</center>

Esto es equivalente a encontrar una transformación que permita proyectar las observaciones sobre ejes que maximicen la varianza de los datos y minimicen la varianza del ruido.

Visto de otra forma, se desea encontrar ejes de proyección de los datos en donde haya la **menor** redundancia (i.e. correlación) posible; es decir, ejes en donde la varianza de la señal sea máxima y su correlación con respecto a otros ejes ("ruido") sea mínima:

<img src="img/COV.png" width="350"/>

Se puede demostrar que estos ejes $\mathbf{w}_j$ están dados por los $k$ vectores propios dominantes (i.e. los asociados con los valores propios más grandes) de la matriz de covarianza de las muestras
<center>
$\mathbf{S}=\sum_{i}(\mathbf{x}_i-\bar{\mathbf{x}})(\mathbf{x}_i-\bar{\mathbf{x}})^{^\textrm{T}}/N$ 
</center>
donde $\bar{\mathbf{x}}$ es la media de las muestras, de tal forma que
<center>
$\mathbf{S}\mathbf{w}_j=\lambda_j\mathbf{w}_j$. 
</center>

Escribiendo $\mathbf{P}=\mathbf{W}^{^\textrm{T}}$, las $k$ componentes principales del vector observado $\mathbf{x}_n$ están dadas por el vector: 
<center>
$\mathbf{p}_n=\mathbf{P}\,(\mathbf{x}_n-\bar{\mathbf{x}})$. 
</center>

De esta forma, las filas de $\mathbf{P}$ son _las componentes principales_ de $\mathbf{X}$ y constituyen una nueva base sobre la que se pueden proyectar los vectores $\mathbf{x}_n$:
<center>
$\mathbf{Y}=\mathbf{P}\,\mathbf{X}$. 
</center>




<img src="img/PCA_3.png" width="400"/>
<em><center>Proyección de la matriz original en el espacio de componentes principales (Tharwat, 2016)</em></center>

# **PCA**

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(svd_solver='auto')

Y_pca = pca.fit_transform(X)

In [ ]:
np.save('datos/Y_pca.npy', Y_pca)

In [ ]:
Y_pca = np.load('datos/Y_pca.npy')

In [ ]:
ypca=wi.get_dataFrame(Y_pca,df)
print(ypca.shape)
ypca.head()

In [ ]:
pca_vr=pca.explained_variance_ratio_
print(pca_vr[:10])

In [ ]:
wi.distribucion_vr(pca_vr)

# **LSA**

**LSA** (Latent Semantic Analysis) utiliza una ponderación de palabras llamada $\textit{tf}$-$\textit{idf}$.

Tf-idf (del inglés Term frequency – Inverse document frequency), frecuencia de término – frecuencia inversa de documento (o sea, la frecuencia de ocurrencia del término en la colección de documentos), es una medida numérica que expresa cuán relevante es una palabra para un documento en una colección. 

Esta medida se utiliza a menudo como un factor de ponderación en la recuperación de información y la minería de texto. El valor tf-idf aumenta proporcionalmente al número de veces que una palabra aparece en el documento, pero es compensada por la frecuencia de la palabra en la colección de documentos, lo que permite manejar el hecho de que algunas palabras son generalmente más comunes que otras. 

$\textit{tf}\,(t,d) = \frac{f(t,d)}{\max\{f(t,d):t\in d\}}$

$\textit{idf}\,(t,D) = \log\frac{|D|}{1+|\{d\in D:t \in d\}|}$

TF-IDF=$\textit{tf}\times \textit{idf}$

LSI utiliza la Descomposición en Valores Singulares (Singular Value Decomposition) o SVD para calcular tres matrices como sigue (Baker, 2005):
<center>
$\mathbf{X}_{nd}=\mathbf{U}_{nn}\,\mathbf{D}_{nd}\,\mathbf{W}^{^\textrm{T}}_{dd}$
</center>

donde $\mathbf{U}\,\mathbf{U}^{^\textrm{T}}=\mathbf{I}$, $\mathbf{V}\,\mathbf{V}^{^\textrm{T}}=\mathbf{I}$; las columnas de $\mathbf{U}$ son los vectores propios ortonormales de $\mathbf{X}\,\mathbf{X}^{^\textrm{T}}$, las columnas de $\mathbf{W}$ son los vectores propios ortonormales de $\mathbf{X}^{^\textrm{T}}\,\mathbf{X}$, y $\mathbf{D}$ es una matriz diagonal que contiene las raíces cuadradas de los vectores propios de $\mathbf{U}$ o $\mathbf{W}$ en orden descendiente.

**Nota** que PCA puede obtenerse a partir de SVD. Una diferencia importante es que SVD no necesita centrar los datos.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

q=300  #Elegimos usar q componentes
svd = TruncatedSVD(n_components=q, n_iter=7, random_state=42)

vectorizer = TfidfVectorizer(vocabulary=vocab_.vocabulary_)

corpus = df.Texto.tolist()
D_tfidf = vectorizer.fit_transform(corpus)
print(D_tfidf[:2])

In [ ]:
dlsa=svd.fit_transform(D_tfidf)

In [4]:
dlsa = np.load('datos/dlsa.npy')

In [5]:
dlsa=wi.get_dataFrame(dlsa,df)
print(dlsa.shape)
dlsa.head()

(4041, 301)


,doc_id,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,1039186,0.121646,-0.024406,-0.039368,-0.035705,-0.041790,-0.010379,-0.004593,-0.042547,-0.021562,...,0.020695,0.008753,-0.013189,0.022894,-0.008763,-0.013317,-0.004110,-0.006685,-0.016296,0.020286
1,1039418,0.196638,-0.058735,-0.067956,-0.072488,-0.109628,0.022077,-0.010523,0.085696,0.116796,...,-0.027488,0.009622,0.027392,0.002523,-0.010163,-0.005532,-0.022401,-0.004529,-0.028139,-0.008621
2,1886236,0.119631,-0.034328,-0.042508,-0.039718,-0.048568,-0.021897,-0.001945,-0.007390,0.024269,...,0.004913,-0.023344,-0.002691,-0.002397,0.011871,0.015983,0.003193,-0.012030,0.007192,0.004401
3,1039204,0.150461,-0.054271,-0.025108,-0.037852,-0.058971,0.035331,-0.031666,0.034916,-0.018692,...,0.003030,0.038374,-0.008059,0.007696,-0.008421,-0.000248,0.010082,0.011281,-0.005485,-0.015180
4,1038384,0.118576,-0.060460,-0.039442,0.000945,0.146879,0.024474,0.013319,-0.022460,0.021937,...,-0.025737,-0.023569,-0.020597,-0.069216,0.064265,0.029761,-0.016103,0.037081,0.018603,0.046333


In [6]:
svd_vr=svd.explained_variance_ratio_
print(svd_vr[:10])

NameError: name 'svd' is not defined

In [ ]:
wi.distribucion_vr(svd_vr)

# **Ejercicio**

Queremos saber que tan bien podemos modelar documentos utilizando estas técnicas de reducción de dimensionalidad.

Observa la distribución de los componentes principales de la matriz BoW, calculados por PCA y por LSI. 

Elige un número $q$ de componentes principales (que llamaremos _representativos_) para PCA y compara la calidad de los documentos más cercanos (semejantes) a los documentos de análisis mostrados aquí abajo.

### Documentos de análisis

In [7]:
docus = df[(df['doc_id']=='1023628') |\
           (df['doc_id']=='1024447') |\
           (df['doc_id']=='1035967') |\
           (df['doc_id']=='1891029') |\
           (df['doc_id']=='1894599') ].\
            drop(columns=['Total','Conteos','Palabras','clase'])  
docus.index=range(len(docus.index))

docus.head()

,doc_id,Texto
0,1891029,accidente trenes chatsworth ángeles accidente ...
1,1023628,revanchismo francés revanche revancha término ...
2,1024447,templo siglo xix situado centro villa pola sie...
3,1035967,ilyushin avión pasajeros largo alcance diseñad...
4,1894599,tomás teresa atleta español nacido santona pro...


### Análisis usando PCA

In [ ]:
q=300  #elige un numero q de componentes principales
pca_rep=wi.get_representativos(ypca,q)
print(pca_rep.shape)
pca_rep.head()

Modelamos los documentos utilizando las componentes principales

In [ ]:
edf_pca=wi.modela_documentos_rep(pca_rep)
print(edf_pca.shape)
edf_pca.head()

Calculamos el vecino más cercano a cada uno de los documentos de análisis

In [ ]:
k=1
vecinos_pca=wi.k_vecinos_mas_cercanos(docus,edf_pca,k)

Por cada documento de análisis, podemos ver qué documento es el más semejante (el vecino más cercano)

In [ ]:
print(vecinos_pca['1891029'])

Traemos los textos correspondientes y comparamos

In [ ]:
test1=df[df['doc_id']=='1891029'].Texto.values[0][:400]
test2=df[df['doc_id']=='1887192'].Texto.values[0][:400]
print(test1)
print(test2)

Podemos repetir el proceso con los demás documentos de análisis

In [ ]:
print(vecinos_pca['1023628'])

In [ ]:
test1=df[df['doc_id']=='1023628'].Texto.values[0][:400]
test2=df[df['doc_id']=='1875751'].Texto.values[0][:400]
print(test1)
print(test2)

### Análisis usando LSA

In [8]:
q=300  #debe ser <= 300 o debes correr de nuevo el algoritmo más arriba
lsa_rep=wi.get_representativos(dlsa,q)
print(lsa_rep.shape)
lsa_rep.head()

(4041, 301)


,doc_id,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,1039186,0.121646,-0.024406,-0.039368,-0.035705,-0.041790,-0.010379,-0.004593,-0.042547,-0.021562,...,0.020695,0.008753,-0.013189,0.022894,-0.008763,-0.013317,-0.004110,-0.006685,-0.016296,0.020286
1,1039418,0.196638,-0.058735,-0.067956,-0.072488,-0.109628,0.022077,-0.010523,0.085696,0.116796,...,-0.027488,0.009622,0.027392,0.002523,-0.010163,-0.005532,-0.022401,-0.004529,-0.028139,-0.008621
2,1886236,0.119631,-0.034328,-0.042508,-0.039718,-0.048568,-0.021897,-0.001945,-0.007390,0.024269,...,0.004913,-0.023344,-0.002691,-0.002397,0.011871,0.015983,0.003193,-0.012030,0.007192,0.004401
3,1039204,0.150461,-0.054271,-0.025108,-0.037852,-0.058971,0.035331,-0.031666,0.034916,-0.018692,...,0.003030,0.038374,-0.008059,0.007696,-0.008421,-0.000248,0.010082,0.011281,-0.005485,-0.015180
4,1038384,0.118576,-0.060460,-0.039442,0.000945,0.146879,0.024474,0.013319,-0.022460,0.021937,...,-0.025737,-0.023569,-0.020597,-0.069216,0.064265,0.029761,-0.016103,0.037081,0.018603,0.046333


In [9]:
edf_lsa=wi.modela_documentos_rep(lsa_rep)
print(edf_lsa.shape)
edf_lsa.head()

(4041, 2)


,doc_id,Vectores
0,1039186,"[0.12164561687616307, -0.024406265620847013, -..."
1,1039418,"[0.19663802423140955, -0.05873545922712045, -0..."
2,1886236,"[0.11963116468228008, -0.03432836785615718, -0..."
3,1039204,"[0.15046060314843715, -0.05427054969379254, -0..."
4,1038384,"[0.11857619280475228, -0.060459988904037684, -..."


In [ ]:
k=1
vecinos_lsa=wi.k_vecinos_mas_cercanos(docus,edf_lsa,k)

In [ ]:
print(vecinos_lsa['1891029'])

In [ ]:
test1=df[df['doc_id']=='1891029'].Texto.values[0][:400]
test2=df[df['doc_id']=='1035967'].Texto.values[0][:400]
print(test1)
print(test2)

In [ ]:
print(vecinos_lsa['1023628'])

In [ ]:
test1=df[df['doc_id']=='1023628'].Texto.values[0][:400]
test2=df[df['doc_id']=='1885462'].Texto.values[0][:400]
print(test1)
print(test2)

<hr/>
<hr/>

# **Matriz palabra-documento**

Queremos ver ahora si podemos representar documentos utilizando representaciones latentes de palabras

Una forma de representar palabras como vectores de co-ocurrencia en documentos es usando LSI sobre la matriz Palabra-Documento:

<img src="img/V.png" width="300"/>

## **LSI de Matriz Palabra-Documento**

**Nota que el algoritmo SVD va a representar cada palabra en un vector de dimensión 300.** 

In [ ]:
V_TFIDF = D_tfidf.T 

vlsa=svd.fit_transform(V_TFIDF)

vlsa = pd.DataFrame(data = vlsa)

print(vlsa.shape)
vlsa.head()

## Índices de las palabras del vocabulario

In [ ]:
vocabulario=vocab_.vocabulary_
vocabulario = OrderedDict(sorted(vocabulario.items(), key=lambda v: v[1]))
print('palabra {:2} índice'.format(''))
for x in list(vocabulario)[:5]:
    print ("{:11}:{:2} ".format(x,vocabulario[x]))

In [ ]:
edf_vlsa=wi.modela_documentos_w(vlsa,df,vocabulario)
print(edf_vlsa.shape)
edf_vlsa.head()

## Test con documentos de análisis

In [ ]:
k=1
vecinos_vlsa=wi.k_vecinos_mas_cercanos(docus,edf_vlsa,k)

In [ ]:
print(vecinos_vlsa['1891029'])

In [ ]:
test1=df[df['doc_id']=='1891029'].Texto.values[0][:200]
test2=df[df['doc_id']=='1894767'].Texto.values[0][:200]
print(test1)
print(test2)

In [ ]:
print(vecinos_vlsa['1023628'])

In [ ]:
test1=df[df['doc_id']=='1023628'].Texto.values[0][:200]
test2=df[df['doc_id']=='1894767'].Texto.values[0][:200]
print(test1)
print(test2)

**Referencias**<br>

Baker, K.. 2005. Singular value decomposition tutorial
The Ohio State University Vol.24

Loper, E. and Bird, S. 2002. NLTK: the Natural Language Toolkit. In _Proceedings of the ACL-02 Workshop on Effective tools and methodologies for teaching natural language processing and computational linguistics - Volume 1 (ETMTNLP '02), Vol. 1_. Association for Computational Linguistics, Stroudsburg, PA, USA, 63-70. DOI: https://doi.org/10.3115/1118108.1118117 

McKinney, W., & others. (2010). Data structures for statistical computing in python. In _Proceedings of the 9th Python in Science Conference_ (Vol. 445, pp. 51–56).

Tharwat, A. (2016). Principal component analysis-a tutorial. IJAPR, 3(3), 197-240.